<a href="https://colab.research.google.com/github/pycriador/PyMalaDireta/blob/main/%5BGoogle_Drive%5D_PyMalaDireta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install docxtpl pandas numpy

In [ ]:
!wget https://libreoffice.soluzioniopen.com/stable/fresh/LibreOffice-fresh.basic-x86_64.AppImage

In [ ]:
import os, re

class appCORE():

    def files(self, path, extension):
        for file in os.listdir(path):
            if os.path.isfile(os.path.join(path, file)):
                if re.search(extension, file):
                    yield file

    def dir_list(self, path=None, extension=None):
        list = []

        for path_info in path:
            dirs = self.files(path_info, extension)
            for dir_info in dirs:
                list.append("%s/%s" % (path_info, dir_info))

        return list

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Criar diretório para exportação
!mkdir "/content/drive/MyDrive/docx"
!mkdir "/content/drive/MyDrive/pdf"

#'drive/MyDrive/PASTA'
app = appCORE()
path = ['drive/MyDrive/docx']
extension = '.docx$'

dir_list = app.dir_list(path=path, extension=extension)

for ffile in dir_list:
  print(f'Lendo o arquivo: {ffile}')

In [ ]:
from docxtpl import DocxTemplate
import pandas as pd
import numpy as np
import time
import subprocess

class appMalaDireta():

    def read_template(self, templates='templates', page=None, encoding='ISO-8859-1'):
        self.template = DocxTemplate(f"{templates}/{page}")

    def read_xlsx(self, file=None, sheet=None):
        xlsx = pd.ExcelFile(file)
        return pd.read_excel(xlsx)

    def create(self, page='', export_name='', templates_path='', encoding='', 
               data=pd.DataFrame(), export_docx=''):
        for index, row in data.iterrows():
            timestamp = int(time.time())
            #Ler HTML como template
            self.read_template(templates=templates_path, page=page, encoding=encoding)
            #Se quiser mudar o nome do arquivo final com info da planilha:
            CPF = ''.join(filter(str.isdigit, row["CPF"]))
            export_name = f'{row["NOME"].upper().strip()}-{CPF}'
            #print(f'Gerando o arquivo: {export_name}')
            #Converter Nan em nada
            dados = row.replace(np.nan, '', regex=True)
            #Converter dados em string
            dados = dados.apply(str)
            self.template.render(dados.str.upper())
            print(f'Gerando o DOCX: {export_docx}/{export_name}-{timestamp}.docx')
            self.template.save(f'{export_docx}/{export_name}-{timestamp}.docx')

In [ ]:
from google.colab import files

In [ ]:
import os, shutil, glob

print('Limpando templates antigos XLSX')
fileList = glob.glob('*.xlsx', recursive=True)

for filePath in fileList:
    try:
        os.remove(filePath)
    except:
        print("Erro ao remover:", filePath)

print('Limpando templates antigos DOCX')
fileList = glob.glob('*.docx', recursive=True)

for filePath in fileList:
    try:
        os.remove(filePath)
    except:
        print("Erro ao remover:", filePath)

print('Limapando arquivos de exemplo')
try:
  shutil.rmtree('sample_data')
except Exception as e:
  print(f'Erro: {e}')

print('Criando pasta para os arquivos')
try:
  os.mkdir('arquivos')
except Exception as e:
  print(f'Erro: {e}')

In [ ]:
print('Escolha o arquivo com os dados em XLSX')
ap_up = files.upload()

for ap in ap_up.keys():
  app = appMalaDireta()
  base = app.read_xlsx(file=ap)

In [ ]:
print('Escolha o arquivo com o template em DOCX')
td_up = files.upload()
for td_up in td_up.keys():
  app.create(export_name='', templates_path='.', page=td_up, encoding='ISO-8859-1', data=base, export_docx='/content/drive/MyDrive/docx')

In [ ]:
import subprocess

!chmod a+x LibreOffice*.AppImage

try:
  subprocess.check_call([f'./LibreOffice-fresh.basic-x86_64.AppImage', '--headless', '--norestore',
                            '--writer', '--convert-to', 'pdf', '--outdir', '/content/drive/MyDrive/pdf',
                            f'/content/drive/MyDrive/TESTE 1-34523123402-1645792308.docx'])
except Exception as e:
  print(e)